In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
ticker = "^BVSP"
data = yf.Ticker(ticker).history(period="2y")

data['next_day_close'] = data['Close'].shift(-1)
data['prev_day_close'] = data['Close'].shift(1)
data['close_5_days_ago'] = data['Close'].shift(5)

data.dropna(inplace=True)

data['price_increase'] = (data['next_day_close'] > data['Close']).astype(int)

data.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)

In [ ]:
train_data = data.iloc[0:250]
test_data = data.iloc[250:500]

In [ ]:
features = ['Close', 'Volume', 'price_increase', 'prev_day_close', 'close_5_days_ago']
X_train = train_data[features]
X_test = test_data[features]
y_train = train_data['next_day_close']
y_test = test_data['next_day_close']

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [ ]:
print(f"Erro Médio Absoluto (MAE): {mae}")
print(f"Erro Quadrático Médio (MSE): {mse}")
print(f"Coeficiente de Determinação (R2): {r2}")
print(f"Erro Percentual Absoluto Médio (MAPE): {mape}%")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test.values, label='Valores Reais')
plt.plot(y_test.index, y_pred, label='Previsões')
plt.xlabel('Data')
plt.ylabel('Valor')
plt.title('Comparação de Valores Reais e Previsões')
plt.legend()
plt.show()

In [ ]:
last_data_points = data.tail(3)
X_future = last_data_points[features]
X_future_scaled = scaler.transform(X_future)
future_predictions = model.predict(X_future_scaled)
print(f"Previsões para os próximos 3 dias:\n{future_predictions}")